In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as tck

In [ ]:
n_dimensions = 2
n_constrains = 5
n_anchors = 10
n_positions = 30
fixed_probability = True
directory = "results/ranks/"

key = "_d{}_c{}_a{}_p{}_{}".format(n_dimensions, n_constrains, n_anchors, n_positions, True)

In order to generate results, run
    
    hypothesis.py

In [ ]:
ranks = np.load(directory+"ranks"+key+".npy")

In [ ]:
DK = n_dimensions * n_constrains
stop = 3 * DK
n_repetitions = ranks.shape[1]

if fixed_probability:
    x = np.linspace(0, 1, ranks.shape[0])
else:
    x = (np.arange(n_positions * n_anchors) + 1) / DK
f, ax = plt.subplots()
plt.plot(
    x[:stop],
    np.mean(ranks, axis=1)[:stop] / DK,
    label="mean rank / full rank")
plt.plot(
    x[:stop],
    np.sum(ranks >= DK, axis=1)[:stop] / n_repetitions,
    label="prob. of full rank")
if fixed_probability:
    plt.xlabel("probabitlty")
else:
    plt.xlabel("number of measurements")
plt.grid()
if not fixed_probability:
    ax.xaxis.set_major_formatter(tck.FormatStrFormatter('%g DK'))
    ax.xaxis.set_major_locator(tck.MultipleLocator(base=1))
plt.legend()
plt.savefig(directory + "ranks_DK" + key + ".pdf")
plt.show()

## Estimation of probability
An attempt to estimate probability of matrix being full rank

In [ ]:
def no_edge_prob(alpha, D, K, M, N):
    exponent = (N - (K - 1)) * (M - (D - 1))
    base = (1 - alpha * (N * D) / (M * N))
    if base <= 0:
        return np.nan
    return np.power(base, exponent)

def simplified_nep(alpha, p, M, N):
    return no_edge_prob(alpha, p * M, p * N, M, N)

def total_prob(alpha, p, M, N):
    p_part = simplified_nep(alpha, p, M, N)
    if p_part == np.nan:
        return np.nan
    exponent = M * N * p * p
    return 1 - np.power((1 - p_part), exponent)

In [ ]:
M = 100
N = 100
a_max = 2
p_max = 1
alphas = np.linspace(0, a_max, 100)

ps = np.linspace(0, p_max, 100)

results = np.zeros((len(alphas), len(ps)))

for i_a, a in enumerate(alphas):
    for i_p, p in enumerate(ps):
        results[i_a, i_p] = total_prob(a, p, M, N)

In [ ]:
plt.matshow(results, extent=[0, p_max, a_max, 0])
plt.colorbar()
plt.show()